In [3]:
import sys
sys.path.append("../")

In [4]:
import pandas as pd
import datetime as dt
import plotly.graph_objects as go
from technicals.indicatorspre import RSI
from technicals.patterns import apply_patterns
from plotting import CandlePlot

In [5]:
df_raw = pd.read_pickle("../data/XAUUSD_H1.pkl")

In [6]:
df_raw.shape

(31263, 13)

In [7]:
df_an = df_raw.copy()#df_raw.iloc[-6000:].copy()
df_an.reset_index(drop=True, inplace=True)

In [8]:
df_an.shape

(31263, 13)

In [9]:
df_an = RSI(df_an)

In [10]:
df_an.tail()

,time,bid_o,bid_h,bid_l,bid_c,ask_o,ask_h,ask_l,ask_c,mid_o,mid_h,mid_l,mid_c,RSI_14
31258,2025-03-13 11:00:00,2944.07,2948.65,2944.07,2946.24,2944.23,2948.73,2944.23,2946.50,2944.150,2948.690,2944.150,2946.370,64.279021
31259,2025-03-13 12:00:00,2946.24,2949.11,2941.02,2942.39,2946.49,2949.48,2941.16,2942.56,2946.365,2949.295,2941.090,2942.475,58.702188
31260,2025-03-13 13:00:00,2942.37,2952.42,2939.11,2950.38,2942.55,2952.57,2939.19,2950.56,2942.460,2952.495,2939.150,2950.470,65.347927
31261,2025-03-13 14:00:00,2950.40,2971.13,2949.86,2969.94,2950.57,2971.31,2950.01,2970.09,2950.485,2971.220,2949.935,2970.015,75.659879
31262,2025-03-13 15:00:00,2969.94,2978.30,2966.43,2977.78,2970.10,2978.55,2966.54,2978.02,2970.020,2978.425,2966.485,2977.900,78.446513


In [11]:
df_an = apply_patterns(df_an)

In [12]:
df_an['EMA_200'] = df_an.mid_c.ewm(span=200, min_periods=200).mean()

In [13]:
df_an.columns

Index(['time', 'bid_o', 'bid_h', 'bid_l', 'bid_c', 'ask_o', 'ask_h', 'ask_l',
       'ask_c', 'mid_o', 'mid_h', 'mid_l', 'mid_c', 'RSI_14', 'direction',
       'body_size', 'full_range', 'body_perc', 'body_lower', 'body_upper',
       'body_bottom_perc', 'body_top_perc', 'mid_point', 'low_change',
       'high_change', 'body_size_change', 'mid_point_prev', 'mid_point_prev_2',
       'body_size_prev', 'direction_prev', 'direction_prev_2',
       'body_perc_prev', 'body_perc_prev_2', 'close_price_prev', 'open_price',
       'close_price', 'HANGING_MAN', 'SHOOTING_STAR', 'SPINNING_TOP',
       'MARUBOZU', 'ENGULFING', 'TWEEZER_TOP', 'TWEEZER_BOTTOM',
       'MORNING_STAR', 'EVENING_STAR', 'PIERCING_LINE', 'PIN_BAR',
       'THREE_WHITE_SOLDIERS', 'EMA_200'],
      dtype='object')

In [14]:
our_cols = ['time', 'mid_o', 'mid_h', 'mid_l', 'mid_c',
                'ask_c','bid_c', 'ENGULFING', 'direction', 'EMA_200', 'RSI_14' ]

In [15]:
df_slim = df_an[our_cols].copy()
df_slim.dropna(inplace=True)
df_slim.reset_index(drop=True, inplace=True)

In [16]:
df_slim.head()

,time,mid_o,mid_h,mid_l,mid_c,ask_c,bid_c,ENGULFING,direction,EMA_200,RSI_14
0,2020-01-14 14:00:00,1544.200,1547.085,1542.520,1546.455,1546.57,1546.34,False,1,1554.180147,51.971579
1,2020-01-14 15:00:00,1546.385,1546.840,1541.045,1543.070,1543.16,1542.98,True,-1,1554.052495,44.349179
2,2020-01-14 16:00:00,1543.075,1544.415,1542.240,1542.440,1542.57,1542.31,False,-1,1553.919276,43.082708
3,2020-01-14 17:00:00,1542.465,1543.910,1542.090,1543.620,1543.74,1543.50,True,1,1553.801301,46.182681
4,2020-01-14 18:00:00,1543.605,1548.030,1542.980,1545.780,1545.91,1545.65,False,1,1553.709558,51.400641


In [17]:
BUY = 1
SELL = -1
NONE = 0
RSI_LIMIT = 50.0

def apply_signal(row):
    if row.ENGULFING == True:
        if row.direction == BUY and row.mid_l > row.EMA_200:
            if row.RSI_14 > RSI_LIMIT:
                return BUY
        if row.direction == SELL and row.mid_h < row.EMA_200:
            if row.RSI_14 < RSI_LIMIT:
                return SELL
    return NONE        

In [18]:
df_slim["SIGNAL"] = df_slim.apply(apply_signal, axis=1)

In [19]:
df_slim["SIGNAL"].value_counts()

SIGNAL
 0    28297
 1     1646
-1     1121
Name: count, dtype: int64

In [20]:
LOSS_FACTOR = -1.0
PROFIT_FACTOR = 1.5

def apply_take_profit(row):
    if row.SIGNAL != NONE:
        return (row.mid_c - row.mid_o) * 1.5 + row.mid_c
    else:
        return 0.0

def apply_stop_loss(row):
    if row.SIGNAL != NONE:
        return row.mid_o
    else:
        return 0.0

In [21]:
df_slim["TP"] = df_slim.apply(apply_take_profit, axis=1)
df_slim["SL"] = df_slim.apply(apply_stop_loss, axis=1)

In [22]:
df_slim[df_slim.SIGNAL==BUY].head()

,time,mid_o,mid_h,mid_l,mid_c,ask_c,bid_c,ENGULFING,direction,EMA_200,RSI_14,SIGNAL,TP,SL
29,2020-01-15 20:00:00,1555.455,1557.075,1554.300,1556.205,1556.30,1556.11,True,1,1553.208798,61.232784,1,1557.330,1555.455
35,2020-01-16 03:00:00,1556.020,1556.815,1555.480,1556.520,1556.61,1556.43,True,1,1553.417529,60.536269,1,1557.270,1556.020
43,2020-01-16 11:00:00,1554.800,1556.500,1554.570,1556.290,1556.40,1556.18,True,1,1553.490274,58.418260,1,1558.525,1554.800
61,2020-01-17 06:00:00,1555.720,1557.015,1554.355,1556.920,1557.03,1556.81,True,1,1553.458130,60.721254,1,1558.720,1555.720
67,2020-01-17 12:00:00,1556.580,1559.790,1556.525,1558.970,1559.11,1558.83,True,1,1553.640734,63.828401,1,1562.555,1556.580


In [23]:
df_plot = df_slim.iloc[70:100]
cp = CandlePlot(df_plot, candles=True)

trades = cp.df_plot[cp.df_plot.SIGNAL != NONE]

markers = ['mid_c', 'TP', 'SL']
marker_colors = ['#0000FF', '#00FF00', '#FF0000']

for i in range(3):
    cp.fig.add_trace(go.Scatter(
        x = trades.sTime,
        y = trades[markers[i]],
        mode = 'markers',
        marker=dict(color=marker_colors[i], size=12)
    ))

cp.show_plot(line_traces=["EMA_200"], sec_traces=['RSI_14'], height=250)

In [24]:
class Trade:
    def __init__(self, row):
        self.running = True
        self.start_index = row.name
        self.start_price = row.mid_c
        self.trigger_price = row.mid_c
        self.SIGNAL = row.SIGNAL
        self.TP = row.TP
        self.SL = row.SL
        self.result = 0.0
        self.end_time = row.time
        self.start_time = row.time
        self.duration = 0
        
    def close_trade(self, row, result, trigger_price):
        self.running = False
        self.result = result
        self.end_time = row.time
        self.trigger_price = trigger_price
        
    def update(self, row):
        self.duration += 1
        if self.SIGNAL == BUY:
            if row.mid_h >= self.TP:
                self.close_trade(row, PROFIT_FACTOR, row.mid_h)
            elif row.mid_l <= self.SL:
                self.close_trade(row, LOSS_FACTOR, row.mid_l)
        if self.SIGNAL == SELL:
            if row.mid_l <= self.TP:
                self.close_trade(row, PROFIT_FACTOR, row.mid_l)
            elif row.mid_h >= self.SL:
                self.close_trade(row, LOSS_FACTOR, row.mid_h)    

In [25]:
open_trades = []
closed_trades = []

for index, row in df_slim.iterrows():
    for ot in open_trades:
        ot.update(row)
        if ot.running == False:
            closed_trades.append(ot)
    open_trades = [x for x in open_trades if x.running == True]
    
    if row.SIGNAL != NONE:
        open_trades.append(Trade(row))    

In [26]:
df_results = pd.DataFrame.from_dict([vars(x) for x in closed_trades])

In [27]:
df_results.result.sum()

np.float64(473.0)

In [28]:
df_results.sort_values(by="start_index", inplace=True)

In [31]:
df_m5 = pd.read_pickle("../data/XAUUSD_M1.pkl")

FileNotFoundError: [Errno 2] No such file or directory: '../data/XAUUSD_M1.pkl'

In [ ]:
df_m5.shape

In [ ]:
df_m5.time.max()

In [ ]:
df_raw.time.max()

In [ ]:
from dateutil import parser

In [ ]:
time_min = parser.parse("2021-12-15T10:00:00")
time_max = parser.parse("2021-12-15T11:00:00")
df_m5_s = df_m5[(df_m5.time>=time_min)&(df_m5.time<=time_max)]
df_raw_s = df_raw[(df_raw.time>=time_min)&(df_raw.time<=time_max)]

In [ ]:
df_m5_s

In [ ]:
df_raw_s

In [ ]:
df_m5_slim = df_m5[['time','mid_h', 'mid_l' ]].copy()

In [ ]:
df_m5_slim.head()

In [ ]:
df_signals = df_slim[df_slim.SIGNAL != NONE].copy() 

In [ ]:
df_signals['m5_start'] = [x + dt.timedelta(hours=1) for x in df_signals.time]

In [ ]:
df_signals['start_index_h1'] = df_signals.index

In [ ]:
df_signals.head()

In [ ]:
df_signals.columns

In [ ]:
df_signals.drop(['time', 'mid_o', 'mid_h', 'mid_l', 'ask_c', 'bid_c',
                'ENGULFING', 'EMA_200', 'RSI_14', 'direction'], axis=1, inplace=True)

In [ ]:
df_signals.head()

In [ ]:
df_signals.rename(columns={
    'mid_c' : 'start_price',
    'm5_start' : 'time'
}, inplace=True)

In [ ]:
df_signals.head(2)

In [ ]:
df_m5_slim.head(2)

In [ ]:
merged = pd.merge(left=df_m5_slim, right=df_signals, on='time', how='left')

In [ ]:
merged.fillna(0, inplace=True)

In [ ]:
merged.SIGNAL = merged.SIGNAL.astype(int)
merged.start_index_h1 = merged.start_index_h1.astype(int)

In [ ]:
merged.head()

In [ ]:
class TradeM5:
    def __init__(self, row):
        self.running = True
        self.start_index_m5 = row.name
        self.start_index_h1 = row.start_index_h1
        self.start_price = row.start_price
        self.trigger_price = row.start_price
        self.SIGNAL = row.SIGNAL
        self.TP = row.TP
        self.SL = row.SL
        self.result = 0.0
        self.end_time = row.time
        self.start_time = row.time
        self.duration = 1
        
    def close_trade(self, row, result, trigger_price):
        self.running = False
        self.result = result
        self.end_time = row.time
        self.trigger_price = trigger_price
        
    def update(self, row):
        self.duration += 1
        if self.SIGNAL == BUY:
            if row.mid_h >= self.TP:
                self.close_trade(row, PROFIT_FACTOR, row.mid_h)
            elif row.mid_l <= self.SL:
                self.close_trade(row, LOSS_FACTOR, row.mid_l)
        if self.SIGNAL == SELL:
            if row.mid_l <= self.TP:
                self.close_trade(row, PROFIT_FACTOR, row.mid_l)
            elif row.mid_h >= self.SL:
                self.close_trade(row, LOSS_FACTOR, row.mid_h)   

In [ ]:
open_trades_m5 = []
closed_trades_m5 = []

for index, row in merged.iterrows():
    for ot in open_trades_m5:
        ot.update(row)
        if ot.running == False:
            closed_trades_m5.append(ot)
    open_trades_m5 = [x for x in open_trades_m5 if x.running == True]
    
    if row.SIGNAL != NONE:
        open_trades_m5.append(TradeM5(row))   

In [ ]:
df_res_m5 = pd.DataFrame.from_dict([vars(x) for x in closed_trades_m5])

In [ ]:
df_res_m5.head()

In [ ]:
df_res_m5.result.sum()